In [15]:
import pandas as pd
import numpy as np
import datasets

/Users/enjalot/code/latent-sae/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file = "/Users/enjalot/latent-scope-data/data-00002-of-00099/input.parquet"

In [11]:
df = pd.read_parquet(file, engine="pyarrow")


In [12]:
df.shape

(260748, 9)

In [34]:
df = df.sample(frac=0.10, random_state=42)

In [35]:
df.shape

(26075, 9)

In [36]:
df["embedding"].apply(lambda x: np.array(x, dtype=np.float16))

25366     [-0.00596, 0.07855, -0.2135, -0.1115, 0.04138,...
29233     [0.06445, 0.0951, -0.2299, 9.26e-05, -0.004925...
168070    [0.06683, 0.1324, -0.1777, -0.05756, -0.02742,...
157460    [0.0591, 0.07855, -0.2041, -0.02621, 0.03262, ...
134131    [0.06995, 0.03262, -0.1782, -0.07776, 0.01059,...
                                ...                        
3448      [0.006435, 0.0475, -0.2489, 0.005424, 0.0793, ...
128571    [0.04968, 0.00584, -0.2075, -0.0279, 0.05637, ...
70280     [0.04373, 0.0661, -0.2108, -0.0513, 0.03702, 0...
47317     [0.05054, 0.0496, -0.1768, -0.01174, 0.05362, ...
22640     [0.0758, 0.08154, -0.1747, -0.0587, 0.05353, 0...
Name: embedding, Length: 26075, dtype: object

In [37]:
train_df = df.sample(frac=0.99, random_state=42)
val_df = df.drop(train_df.index)

train_df.to_parquet("./data/test_train.parquet", engine='pyarrow', index=False, compression='snappy')
val_df.to_parquet("./data/test_val.parquet", engine='pyarrow', index=False, compression='snappy')


In [38]:
train_df.shape

(25814, 9)

In [39]:
val_df.shape

(261, 9)

In [50]:
# Convert pandas DataFrame to datasets Dataset
ds = datasets.Dataset.from_pandas(df)


In [51]:
# ds = datasets.load_dataset("parquet", data_files=file)


In [54]:
dss = ds.train_test_split(test_size=0.01, seed=42)

In [62]:
dss.save_to_disk("./data/test_train", format="pyarrow")

#dss.save_to_disk("./data/test_train")

TypeError: DatasetDict.save_to_disk() got an unexpected keyword argument 'format'

In [65]:
dst = datasets.load_dataset("arrow", data_dir="./data/test_train", streaming=True)

In [68]:
i = 0
try:
  for item in dst["test"]:
      # print(item["embedding"])
      i+=1
      print(i)
except:
    print("except i", i)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261


Failed to read file '/Users/enjalot/code/latent-sae/notebooks/data/test_train/test/state.json' with error <class 'pyarrow.lib.ArrowInvalid'>: Expected to read 538970747 metadata bytes, but only read 243


except i 261


In [59]:
dst['train']

IterableDataset({
    features: ['chunk_index', 'chunk_text', 'chunk_tokens', 'chunk_token_count', 'id', 'url', 'score', 'dump', 'embedding', '__index_level_0__'],
    n_shards: 2
})

In [60]:
dst["test"]

IterableDataset({
    features: ['chunk_index', 'chunk_text', 'chunk_tokens', 'chunk_token_count', 'id', 'url', 'score', 'dump', 'embedding', '__index_level_0__'],
    n_shards: 2
})